<a href="https://colab.research.google.com/github/Thabet-Chaaouri/Efficient_fine_tuning/blob/main/Prefix_VS_Basic_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers peft accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Setup

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, get_linear_schedule_with_warmup, default_data_collator
from peft import PrefixTuningConfig, get_peft_model, TaskType, PeftConfig, PeftModel
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import torch

model_name  = "t5-large"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 1e-2
num_epochs = 5
batch_size =8

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
device

device(type='cuda')

#Load & prepocess dataset

In [ ]:
dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size = 0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
classes = dataset["train"].features["label"].names
dataset = dataset.map(lambda examples : {label_column : [classes[label] for label  in examples["label"]]} ,batched=True, num_proc=1)

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
def preprocess_function(examples):
  inputs =examples[text_column]
  targets =examples[label_column]
  model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")
  labels = tokenizer(targets, max_length=2, padding ="max_length", truncation =True, return_tensors="pt")
  labels = labels["input_ids"]
  labels[labels ==tokenizer.pad_token_id ] = -100
  model_inputs["labels"] = labels
  return model_inputs

In [ ]:
processed_datasets  = dataset.map(preprocess_function, batched=True, num_proc=1, remove_columns=dataset["train"].column_names, 
                                load_from_cache_file = False, desc = "Running tokenizer on dataset")

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [ ]:
train_dataset = processed_datasets["train"]
val_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
eval_dataloader = DataLoader(val_dataset, shuffle=False, collate_fn=default_data_collator, batch_size = batch_size, pin_memory=True)

# Train model with PEFT

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=30)
model = get_peft_model(model, peft_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 1474560 || all params: 739142656 || trainable%: 0.19949599553350633


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(optimizer = optimizer , num_warmup_steps = 0, num_training_steps = len(train_dataloader)*num_epochs)

In [ ]:
model = model.to(device);

for epoch in range(num_epochs):
  model.train()
  total_loss = 0
  for step, batch in enumerate(tqdm(train_dataloader)):
    batch = {k:v.to(device) for k,v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    total_loss += loss.detach().float()
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()

  model.eval()
  eval_loss=0
  eval_preds=[]
  for step, batch in enumerate(tqdm(eval_dataloader)):
    batch = {k : v.to(device) for k,v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)
    loss = outputs.loss
    eval_loss +=loss
    eval_preds.extend(tokenizer.batch_decode(torch.argmax(outputs.logits,-1).detach().cpu().numpy(), skip_special_tokens=True))
  
  eval_epoch_loss = eval_loss/len(eval_dataloader)
  eval_ppl = torch.exp(eval_epoch_loss)
  train_epoch_loss = total_loss/len(train_dataloader)
  train_ppl = torch.exp(train_epoch_loss)
  print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 29/29 [00:06<00:00,  4.43it/s]


epoch=0: train_ppl=tensor(5.3161, device='cuda:0') train_epoch_loss=tensor(1.6707, device='cuda:0') eval_ppl=tensor(1.0958, device='cuda:0') eval_epoch_loss=tensor(0.0915, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.33it/s]


epoch=1: train_ppl=tensor(1.1265, device='cuda:0') train_epoch_loss=tensor(0.1191, device='cuda:0') eval_ppl=tensor(1.0591, device='cuda:0') eval_epoch_loss=tensor(0.0575, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.39it/s]


epoch=2: train_ppl=tensor(1.0823, device='cuda:0') train_epoch_loss=tensor(0.0791, device='cuda:0') eval_ppl=tensor(1.0483, device='cuda:0') eval_epoch_loss=tensor(0.0471, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.39it/s]


epoch=3: train_ppl=tensor(1.0699, device='cuda:0') train_epoch_loss=tensor(0.0675, device='cuda:0') eval_ppl=tensor(1.0504, device='cuda:0') eval_epoch_loss=tensor(0.0492, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.40it/s]

epoch=4: train_ppl=tensor(1.0605, device='cuda:0') train_epoch_loss=tensor(0.0587, device='cuda:0') eval_ppl=tensor(1.0418, device='cuda:0') eval_epoch_loss=tensor(0.0410, device='cuda:0')


In [ ]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

accuracy=97.79735682819384 % on the evaluation dataset
eval_preds[:10]=['positive', 'neutral', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'neutral', 'neutral', 'neutral']
dataset['validation']['text_label'][:10]=['positive', 'neutral', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'neutral', 'neutral', 'neutral']


#Basic Training

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(optimizer = optimizer , num_warmup_steps = 0, num_training_steps = len(train_dataloader)*num_epochs)

In [ ]:
#model = model.to(device);
num_epochs=1
batch_size =8

from accelerate import Accelerator

accelerator = Accelerator(mixed_precision="fp16")
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

for epoch in range(num_epochs):
  model.train()
  total_loss = 0
  for step, batch in enumerate(tqdm(train_dataloader)):
    batch = {k:v.to(device) for k,v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    accelerator.backward(loss)
    #loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    total_loss += loss.detach().float()

  model.eval()
  eval_loss=0
  eval_preds=[]
  for step, batch in enumerate(tqdm(eval_dataloader)):
    batch = {k : v.to(device) for k,v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)
    loss = outputs.loss
    eval_loss +=loss
    eval_preds.extend(tokenizer.batch_decode(torch.argmax(outputs.logits,-1).detach().cpu().numpy(), skip_special_tokens=True))
  
  eval_epoch_loss = eval_loss/len(eval_dataloader)
  eval_ppl = torch.exp(eval_epoch_loss)
  train_epoch_loss = total_loss/len(train_dataloader)
  train_ppl = torch.exp(train_epoch_loss)
  print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 29/29 [00:02<00:00, 10.33it/s]

epoch=0: train_ppl=tensor(nan, device='cuda:0') train_epoch_loss=tensor(nan, device='cuda:0') eval_ppl=tensor(nan, device='cuda:0') eval_epoch_loss=tensor(nan, device='cuda:0')


Out of memory with a batch size of 8 and even 4.

I had to use an accelerator with mixed precision fp16 to fit the model.


#Sharing model and inference

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
peft_model_id = "Thabet/t5-large_PREFIX_TUNING_SEQ2SEQ"
#model.push_to_hub("Thabet/t5-large_PREFIX_TUNING_SEQ2SEQ", use_auth_token=True)

In [ ]:
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
inputs = tokenizer(
    "The Lithuanian beer market made up 14.41 million liters in January , a rise of 0.8 percent from the year-earlier figure , the Lithuanian Brewers ' Association reporting citing the results from its members .",
    return_tensors="pt",
)

In [ ]:
model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['positive']
